### The Tweets

This notebook is for cleaning the tweets and tokenizing with a subsequent analysis.


Actually, these guys: https://link.springer.com/chapter/10.1007/978-3-319-09339-0_62 argue that lemmatization and stemming lead to worse performance in terms of sentiment analysis, while features reservation, negation transformation and repeated letters normalization improves it. They also get better results when by using bigrams and emotions features.


https://www.sciencedirect.com/science/article/pii/S1877050913001385

Also: Stop Word removal might not be necessary. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import swifter
import pickle

# Text mining packages
import spacy
from bs4 import BeautifulSoup
import nltk

# Import cleaning function
from utils.cleaning import clean_tweet, has_arabic, has_cyrillic


/home/adam/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/adam/anaconda3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Loading tweets and user data:

We've already cleaned the tweets a bit checking for user locations and so on. Now it's time to clean the texts, tokenize, remove stop words and so on.

```clean_tweet``` does a bunch of stuff, including removal of retweets, hyperlinks, emojis and so on.

We're removing retweets because we're interested in the "content-creators"

In [3]:
tweets = pd.read_pickle("../data/tweets_clean.pickle")
users = pd.read_pickle("../data/user_clean.pickle")

# Clean
tweets["tweet_text"] = tweets["tweet_text"].swifter.apply(clean_tweet)

# Save it, just to make sure nothing gets lost
with open('../data/tweets_preprocessed.pickle', 'wb') as f:
    pickle.dump(tweets, f)

After cleaning, we can tokenize. Hashtags should be helpful with getting meaningful embeddings since they contain information about the topic. We could later check whether there are differences in embeddings / clusters with or without hashtags. We also remove stopwords using nltk.

In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [5]:
# Set up tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Set stop words
stop_words = set(stopwords.words('english')) 

# Tokenize
tweets_tokenized = [tokenizer.tokenize(t) for t in tweets["tweet_text"]]

# Remove stopwords
tweets_tokenized_noSwords = [[w for w in tweet_tokens if w not in stop_words] for tweet_tokens in tweets_tokenized]

In [6]:
with open('../data/tweets_tokenized.pickle', 'wb') as f:
    pickle.dump(tweets_tokenized_noSwords, f)